In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END

In [6]:
class OverallState(TypedDict):
    user_name: str
    user_age:int
    user_score:float
    user_grade:str

class InputState(TypedDict):
    user_name: str
    user_age:int
    
class OutputState(TypedDict):
    user_name: str
    user_score:float
    user_grade:str
    
class PrivateState(TypedDict):
    user_id:str

In [8]:
def node1(state:InputState)->OverallState:
    return {"user_grade":100}

def node2(state:OverallState)->PrivateState:
    return {"user_id":12345}

def node3(state:PrivateState)->OutputState:
    return {"user_name":"John Doe"}

In [9]:
graph_builder = StateGraph(OverallState,input=InputState,output=OutputState)

graph_builder.add_node("node1",node1)
graph_builder.add_node("node2",node2)
graph_builder.add_node("node3",node3)

graph_builder.add_edge(START, "node1")
graph_builder.add_edge("node1","node2")
graph_builder.add_edge("node2","node3")
graph_builder.add_edge("node3",END)

graph = graph_builder.compile()

In [10]:
result = graph.invoke({'user_name':'lee'})

In [11]:
result

{'user_name': 'John Doe', 'user_grade': 100}

In [17]:
# default stream_mode is updates
for chunk in graph.stream({'user_name':'lee'}, stream_mode="updates"):
    print(chunk)

{'node1': {'user_grade': 100}}
{'node2': {'user_id': 12345}}
{'node3': {'user_name': 'John Doe'}}


In [19]:
for chunk in graph.stream({'user_name':'lee'}, stream_mode="values"):
    print(chunk)

{'user_name': 'lee'}
{'user_name': 'lee', 'user_grade': 100}
{'user_name': 'lee', 'user_grade': 100, 'user_id': 12345}
{'user_name': 'John Doe', 'user_grade': 100, 'user_id': 12345}
